In [2]:
import pandas as pd

In [6]:
pdbbind_df = pd.read_csv('pdbbind_dataset_rna/pdbbind_rna_labels_split.csv', keep_default_na=False)

In [147]:
import os

def generate_ligand_dataframe(folder_path):
    # Initialize an empty list to store data
    data = []
    
    # Iterate through all files in the specified folder
    for filename in os.listdir(folder_path):
        # Only process files that end with ".cif"
        if filename.endswith(".cif"):
            # Split the filename by underscores to extract pdb_id, ligand_id, and ligand_chain
            parts = filename.split('_')
            if len(parts) == 3:
                pdb_id = parts[0]
                ligand_id = parts[1]
                ligand_chain = parts[2].replace(".cif", "")
                
                # Append the extracted information to the data list
                data.append([pdb_id, ligand_id, ligand_chain])

    # Create a DataFrame from the data list
    df = pd.DataFrame(data, columns=['pdb_id', 'ligand_id', 'ligand_chain'])
    
    return df

In [148]:
pdbbind_df_ = generate_ligand_dataframe('/home/tzutang.lin/blue_ufhpc/CoPRA/CoRSA/pdbbind_dataset_rna/parsed_ligand')

In [149]:
import gemmi

cif_file_folder = '/home/tzutang.lin/blue_ufhpc/CoPRA/CoRSA/pdbbind_dataset_rna/complex_database'
for i, row in pdbbind_df_.iterrows():
        pdb_id = row['pdb_id']
        cif_file_path = os.path.join(cif_file_folder, f'{pdb_id}.cif')
        if os.path.exists(cif_file_path):
            doc = gemmi.cif.read(cif_file_path)
            block = doc.sole_block()
            atom_table = block.find(['_atom_site.label_comp_id', '_atom_site.label_asym_id', '_atom_site.auth_asym_id'])
            
            for atom_row in atom_table:
                res_id, seg_chain, auth_chain = atom_row[0], atom_row[1], atom_row[2]
                if (row['ligand_id'] == res_id and row['ligand_chain'] == seg_chain):
                    pdbbind_df_.at[i, 'rna_chain'] = auth_chain
                    break


In [150]:
pdbbind_df_ = pdbbind_df_.drop_duplicates(subset=['pdb_id'])
pdbbind_df_.reset_index(drop=True)

,pdb_id,ligand_id,ligand_chain,rna_chain
0,4ts2,38E,H,Y
1,3b31,IRI,D,A
2,6ck4,G4P,WA,D
3,4yb0,C2E,D,R
4,4ts0,38E,R,Y
...,...,...,...,...
113,3g4m,2BP,B,A
114,4nyc,SVN,B,A
115,3jq4,LC2,C,A
116,3mur,C2E,C,R


In [151]:
import os
import gemmi
import pandas as pd

# Function to extract RNA chain sequence from CIF file
def extract_rna_chain_sequence(cif_file_path, rna_chain):
    try:
        # Load the CIF file
        doc = gemmi.cif.read(cif_file_path)
        block = doc.sole_block()

        # Extract the list of struct asym ids
        struct_asym_table = block.find(['__struct_asym.id'])
        asym_ids = [row[0] for row in struct_asym_table]

        # Find the order of the specified rna_chain
        given_chain_order = None
        for i, asym_id in enumerate(asym_ids):
            if asym_id == rna_chain:
                given_chain_order = i + 1  # Order starts from 1
                break

        # Set to first chain order if rna_chain is not found
        if given_chain_order is None:
            print(f"RNA chain '{rna_chain}' not found in {cif_file_path}. Using the first chain '{asym_ids[0]}' instead.")
            given_chain_order = 1
            rna_chain = asym_ids[0]  # Update rna_chain to the first chain id

        # Extract the RNA sequence for the given chain
        entity_poly_seq_table = block.find(['_entity_poly_seq.num', '_entity_poly_seq.mon_id'])
        seq_nums = [row[0] for row in entity_poly_seq_table]
        mon_ids = [row[1] for row in entity_poly_seq_table]

        rna_chains = {}
        chain_order = 0
        sequence = ""

        for seq_num, mon_id in zip(seq_nums, mon_ids):
            if seq_num == '1':
                # Increment chain order for every new chain start
                chain_order += 1
                sequence = ""  # Start a new sequence for the new chain

            # Append monomer ID to the current chain sequence
            sequence += mon_id
            rna_chains[chain_order] = sequence

        # Return the sequence and updated chain id for the specified chain order
        return rna_chains[given_chain_order] if given_chain_order in rna_chains else None, rna_chain

    except Exception as e:
        # Handle cases where the CIF file does not exist or cannot be parsed
        print(f"Error processing {cif_file_path}: {e}")
        return None, rna_chain


# Set up the path to your CIF files
cif_file_folder = '/home/tzutang.lin/blue_ufhpc/CoPRA/CoRSA/pdbbind_dataset_rna/parsed_rna3db_rnas_whole'

# Iterate over the DataFrame and extract the RNA sequences
rna_sequences = []
updated_rna_chains = []

for i, row in pdbbind_df_.iterrows():
    pdb_id = row['pdb_id']
    rna_chain = row['rna_chain']
    cif_file_path = os.path.join(cif_file_folder, f'{pdb_id}_rna.cif')
    
    # Extract RNA sequence and possibly updated RNA chain
    rna_sequence, updated_rna_chain = extract_rna_chain_sequence(cif_file_path, rna_chain)

    # If the RNA chain was updated, print a notification
    if updated_rna_chain != rna_chain:
        print(f"Updated rna_chain for pdb_id '{pdb_id}' from '{rna_chain}' to '{updated_rna_chain}'.")

    rna_sequences.append(rna_sequence)
    updated_rna_chains.append(updated_rna_chain)

# Add the extracted RNA sequences and updated RNA chains to the DataFrame
pdbbind_df_.loc[:, 'rna_sequence'] = rna_sequences
pdbbind_df_.loc[:, 'rna_chain'] = updated_rna_chains



RNA chain 'B' not found in /home/tzutang.lin/blue_ufhpc/CoPRA/CoRSA/pdbbind_dataset_rna/parsed_rna3db_rnas_whole/1qd3_rna.cif. Using the first chain 'A' instead.
Updated rna_chain for pdb_id '1qd3' from 'B' to 'A'.


In [152]:
pdbbind_df_

,pdb_id,ligand_id,ligand_chain,rna_chain,rna_sequence
0,4ts2,38E,H,Y,GCAGCCGGCUUGUUGAGUAGAGUGUGAGCUCCGUAACUGGUCGCGUC
1,3b31,IRI,D,A,GGUUAUUCAGAUUAGGUAGUCGAAUGACC
2,6ck4,G4P,WA,D,GUGAAAGUGUACCUAGGGUUCCAGCCUAUUUGUAGGUGUUCGGACC...
3,4yb0,C2E,D,R,GUACACGACAAUACUAAACCAUCCGCGAGGAUGGGGCGGAAAGCCU...
4,4ts0,38E,R,Y,GCAGCCGGCUUGUUGAGUAGAGUGUGAGCUCCGUAACUGGUCGCGUC
...,...,...,...,...,...
158,3g4m,2BP,B,A,GGACAUAUAAUCGCGUGGAUAUGGCACGCAAGUUUCUACCGGGCAC...
159,4nyc,SVN,B,A,GCGACUCGGGGUGCCCUUCUGCGUGAAGGCUGAGAAAUACCCGUAU...
160,3jq4,LC2,C,A,GGUCAAGAUAGUAAGGGUCCACGGUGGAUGCCCUGGCGCUGGAGCC...
161,3mur,C2E,C,R,GGUCACGCACAGGGCAAACCAUUCGAAAGAGUGGGACGCAAAGCCU...


In [161]:
import os
from openbabel import openbabel
# Set up the path to your CIF files
ligand_pdb_folder = '/home/tzutang.lin/blue_ufhpc/CoPRA/CoRSA/pdbbind_dataset_rna/parsed_ligand_pdb'

import warnings
from openbabel import openbabel
import os

# Function to extract SMILES and handle potential issues
def extract_ligand_smiles(row):
    pdb_id = row['pdb_id']
    ligand_id = row['ligand_id']
    ligand_chain = row['ligand_chain']

    pdb_file_path = os.path.join(ligand_pdb_folder, f'{pdb_id}_{ligand_id}_{ligand_chain}.pdb')

    if not os.path.exists(pdb_file_path):
        print(f"Warning: PDB file {pdb_file_path} not found.")
        return None

    try:
        with warnings.catch_warnings():
            warnings.simplefilter("ignore", category=UserWarning)
            ligand_smiles = extract_smiles_from_pdb(pdb_file_path)
    except Exception as e:
        print(f"Error extracting SMILES from {pdb_file_path}: {e}")
        ligand_smiles = None

    return ligand_smiles

# Apply the function to the DataFrame
pdbbind_df_['ligand_smiles'] = pdbbind_df_.apply(extract_ligand_smiles, axis=1)



*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is /home/tzutang.lin/blue_ufhpc/CoPRA/CoRSA/pdbbind_dataset_rna/parsed_ligand_pdb/6hmo_GDZ_C.pdb)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is /home/tzutang.lin/blue_ufhpc/CoPRA/CoRSA/pdbbind_dataset_rna/parsed_ligand_pdb/6e8t_HZG_L.pdb)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is /home/tzutang.lin/blue_ufhpc/CoPRA/CoRSA/pdbbind_dataset_rna/parsed_ligand_pdb/5kpy_4PQ_B.pdb)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is /home/tzutang.lin/blue_ufhpc/CoPRA/CoRSA/pdbbind_dataset_rna/parsed_ligand_pdb/6c63_EKJ_K.pdb)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is /home/tzuta

In [167]:
pdbbind_df_

,pdb_id,ligand_id,ligand_chain,rna_chain,rna_sequence,rna_smiles
0,4ts2,38E,H,Y,GCAGCCGGCUUGUUGAGUAGAGUGUGAGCUCCGUAACUGGUCGCGUC,Fc1c(O)c(cc(c1)/C=C\1/C(=O)N(C)C(=N1)C)F
1,3b31,IRI,D,A,GGUUAUUCAGAUUAGGUAGUCGAAUGACC,[Ir](N)(N)(N)(N)(N)N
2,6ck4,G4P,WA,D,GUGAAAGUGUACCUAGGGUUCCAGCCUAUUUGUAGGUGUUCGGACC...,P(=O)(O)(O)O[P@@](=O)(O)OC[C@@H]1O[C@H]([C@H](...
3,4yb0,C2E,D,R,GUACACGACAAUACUAAACCAUCCGCGAGGAUGGGGCGGAAAGCCU...,[P@]1(=O)(O)OC[C@@H]2O[C@H]([C@@H]([C@H]2O[P@@...
4,4ts0,38E,R,Y,GCAGCCGGCUUGUUGAGUAGAGUGUGAGCUCCGUAACUGGUCGCGUC,Fc1c(O)c(cc(c1)/C=C\1/C(=O)N(C)C(=N1)C)F
...,...,...,...,...,...,...
158,3g4m,2BP,B,A,GGACAUAUAAUCGCGUGGAUAUGGCACGCAAGUUUCUACCGGGCAC...,n1cnc2-c1[nH]c(nc2)N
159,4nyc,SVN,B,A,GCGACUCGGGGUGCCCUUCUGCGUGAAGGCUGAGAAAUACCCGUAU...,c1sc2nccnc2c1N
160,3jq4,LC2,C,A,GGUCAAGAUAGUAAGGGUCCACGGUGGAUGCCCUGGCGCUGGAGCC...,C1(=O)O[C@H]2C[C@H](O)C=CC(=CC[C@@H](O)/C=C/C(...
161,3mur,C2E,C,R,GGUCACGCACAGGGCAAACCAUUCGAAAGAGUGGGACGCAAAGCCU...,[P@]1(=O)(O)OC[C@H]2O[C@@H]([C@H]([C@@H]2O[P@]...


In [168]:
pdbbind_df

,pdb,label,group,fold,set
0,1arj,3.000000,4,5,train
1,1byj,5.698970,15,5,train
2,1f27,6.000000,22,5,train
3,1f1t,7.397940,18,3,train
4,1qd3,5.229148,4,5,train
...,...,...,...,...,...
113,6hag,5.431798,0,2,test
114,6p2h,7.468521,12,1,train
115,6e81,5.522879,14,2,valid
116,6e8u,8.337242,34,3,train


In [169]:
merged_df = pd.merge(pdbbind_df_, pdbbind_df,left_on='pdb_id', right_on='pdb', how='inner') 
merged_df = merged_df.drop(columns=['pdb'])

In [173]:
merged_df.to_csv('pdbbind_dataset_rna/pdbbind_rna_processed_index.csv', index=False)

In [177]:
merged_df

,pdb_id,ligand_id,ligand_chain,rna_chain,rna_sequence,rna_smiles,label,group,fold,set
0,4ts2,38E,H,Y,GCAGCCGGCUUGUUGAGUAGAGUGUGAGCUCCGUAACUGGUCGCGUC,Fc1c(O)c(cc(c1)/C=C\1/C(=O)N(C)C(=N1)C)F,6.275724,9,3,valid
1,3b31,IRI,D,A,GGUUAUUCAGAUUAGGUAGUCGAAUGACC,[Ir](N)(N)(N)(N)(N)N,8.698970,24,1,train
2,6ck4,G4P,WA,D,GUGAAAGUGUACCUAGGGUUCCAGCCUAUUUGUAGGUGUUCGGACC...,P(=O)(O)(O)O[P@@](=O)(O)OC[C@@H]1O[C@H]([C@H](...,5.744727,0,2,test
3,4yb0,C2E,D,R,GUACACGACAAUACUAAACCAUCCGCGAGGAUGGGGCGGAAAGCCU...,[P@]1(=O)(O)OC[C@@H]2O[C@H]([C@@H]([C@H]2O[P@@...,6.031517,2,5,train
4,4ts0,38E,R,Y,GCAGCCGGCUUGUUGAGUAGAGUGUGAGCUCCGUAACUGGUCGCGUC,Fc1c(O)c(cc(c1)/C=C\1/C(=O)N(C)C(=N1)C)F,6.275724,9,3,valid
...,...,...,...,...,...,...,...,...,...,...
113,3g4m,2BP,B,A,GGACAUAUAAUCGCGUGGAUAUGGCACGCAAGUUUCUACCGGGCAC...,n1cnc2-c1[nH]c(nc2)N,5.356547,12,1,train
114,4nyc,SVN,B,A,GCGACUCGGGGUGCCCUUCUGCGUGAAGGCUGAGAAAUACCCGUAU...,c1sc2nccnc2c1N,3.552842,5,5,train
115,3jq4,LC2,C,A,GGUCAAGAUAGUAAGGGUCCACGGUGGAUGCCCUGGCGCUGGAGCC...,C1(=O)O[C@H]2C[C@H](O)C=CC(=CC[C@@H](O)/C=C/C(...,5.000000,0,2,test
116,3mur,C2E,C,R,GGUCACGCACAGGGCAAACCAUUCGAAAGAGUGGGACGCAAAGCCU...,[P@]1(=O)(O)OC[C@H]2O[C@@H]([C@H]([C@@H]2O[P@]...,7.823909,2,5,train


In [174]:
general_df = pd.read_csv('general_dataset/general_index.csv', keep_default_na=False)

In [178]:
general_df = general_df.rename(columns={'pdbid': 'pdb_id'})
general_df = general_df[['pdb_id', 'ligand_id', 'ligand_chain', 'rna_chain']]

In [179]:
general_df

,pdb_id,ligand_id,ligand_chain,rna_chain
0,1aju,ARG,B,A
1,1akx,ARG,B,A
2,1am0,AMP,B,A
3,1arj,ARG,B,N
4,1eht,TEP,B,A
...,...,...,...,...
1385,6yl5,SAH,NA,K
1386,6ymi,AMP,Z,O
1387,6ymj,ADN,AA,O
1388,7tql,5GP,NA,3


In [181]:
# Set up the path to your CIF files
cif_file_folder = '/home/tzutang.lin/blue_ufhpc/CoPRA/CoRSA/general_dataset/parsed_rna3db_rnas_whole'

# Iterate over the DataFrame and extract the RNA sequences
rna_sequences = []
updated_rna_chains = []

for i, row in general_df.iterrows():
    pdb_id = row['pdb_id']
    rna_chain = row['rna_chain']
    cif_file_path = os.path.join(cif_file_folder, f'{pdb_id}_rna.cif')
    
    # Extract RNA sequence and possibly updated RNA chain
    rna_sequence, updated_rna_chain = extract_rna_chain_sequence(cif_file_path, rna_chain)

    # If the RNA chain was updated, print a notification
    if updated_rna_chain != rna_chain:
        print(f"Updated rna_chain for pdb_id '{pdb_id}' from '{rna_chain}' to '{updated_rna_chain}'.")

    rna_sequences.append(rna_sequence)
    updated_rna_chains.append(updated_rna_chain)

# Add the extracted RNA sequences and updated RNA chains to the DataFrame
general_df.loc[:, 'rna_sequence'] = rna_sequences
general_df.loc[:, 'rna_chain'] = updated_rna_chains

In [184]:
# Apply the function to the DataFrame
ligand_pdb_folder = '/home/tzutang.lin/blue_ufhpc/CoPRA/CoRSA/general_dataset/parsed_ligands_pdb'

# Function to extract SMILES and handle potential issues
def extract_ligand_smiles(row):
    pdb_id = row['pdb_id']
    ligand_id = row['ligand_id']
    ligand_chain = row['ligand_chain']

    pdb_file_path = os.path.join(ligand_pdb_folder, f'{pdb_id}_{ligand_chain}_{ligand_id}.pdb')

    if not os.path.exists(pdb_file_path):
        print(f"Warning: PDB file {pdb_file_path} not found.")
        return None

    try:
        with warnings.catch_warnings():
            warnings.simplefilter("ignore", category=UserWarning)
            ligand_smiles = extract_smiles_from_pdb(pdb_file_path)
    except Exception as e:
        print(f"Error extracting SMILES from {pdb_file_path}: {e}")
        ligand_smiles = None

    return ligand_smiles

general_df['ligand_smiles'] = general_df.apply(extract_ligand_smiles, axis=1)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is /home/tzutang.lin/blue_ufhpc/CoPRA/CoRSA/general_dataset/parsed_ligands_pdb/1f1t_J_ROS.pdb)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is /home/tzutang.lin/blue_ufhpc/CoPRA/CoRSA/general_dataset/parsed_ligands_pdb/1fmn_B_FMN.pdb)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is /home/tzutang.lin/blue_ufhpc/CoPRA/CoRSA/general_dataset/parsed_ligands_pdb/2cky_I_TPP.pdb)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is /home/tzutang.lin/blue_ufhpc/CoPRA/CoRSA/general_dataset/parsed_ligands_pdb/2cky_R_TPP.pdb)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is /home/tzutang.lin/blue_

In [185]:
general_df

,pdb_id,ligand_id,ligand_chain,rna_chain,rna_sequence,rna_smiles
0,1aju,ARG,B,A,GGCCAGAUUGAGCCUGGGAGCUCUCUGGCC,[NH3][C@@H](C=O)CCCNC(=[NH2])N
1,1akx,ARG,B,A,GGCCAGAUUGAGCCUGGGAGCUCUCUGGCC,[NH3][C@H](C=O)CCCNC(=[NH2])N
2,1am0,AMP,B,A,GGGUUGGGAAGAAACUGUGGCACUUCGGUGCCAGCAACCC,[P@@](=O)(O)(O)OC[C@H]1O[C@@H]([C@@H]([C@@H]1O...
3,1arj,ARG,B,N,GGCAGAUCUGAGCCUGGGAGCUCUCUGCC,[NH3][C@H](C(=O)O)CCCNC(=[NH2])N
4,1eht,TEP,B,A,GGCGAUACCAGCCGAAAGGCCCUUGGCAGCGUC,Cn1c(=O)n(C)c2c(c1=O)[nH]cn2
...,...,...,...,...,...,...
1385,6yl5,SAH,NA,K,GGUCACAACGGCUUCCUGGCGUGACCAUUGGAGCA,N[C@H](CCSC[C@H]1O[C@@H]([C@@H]([C@H]1O)O)n1cn...
1386,6ymi,AMP,Z,O,GGUCACAACGGCUUCCUGGCGUGACC,P(=O)(O)(O)OC[C@@H]1O[C@H]([C@@H]([C@H]1O)O)n1...
1387,6ymj,ADN,AA,O,GGUCACAACGGCUUCCUGGCGUGACC,OC[C@H]1O[C@@H]([C@H]([C@H]1O)O)n1cnc2c(N)ncnc12
1388,7tql,5GP,NA,3,AGCAGAGUGGCGCAGCGGAAGCGUGCUGGGCCCAUAACCCAGAGGU...,P(=O)(=O)OC[C@@H]1O[C@@H]([C@H]([C@@H]1O)O)n1c...


In [190]:
general_df.to_csv('general_dataset/general_processed_index.csv', index=False)

In [39]:

general_df = pd.read_csv('general_dataset/general_processed_index.csv', keep_default_na=False)

In [4]:
general_df

,pdb_id,ligand_id,ligand_chain,rna_chain,rna_sequence,ligand_smiles
0,1aju,ARG,B,A,GGCCAGAUUGAGCCUGGGAGCUCUCUGGCC,[NH3][C@@H](C=O)CCCNC(=[NH2])N
1,1akx,ARG,B,A,GGCCAGAUUGAGCCUGGGAGCUCUCUGGCC,[NH3][C@H](C=O)CCCNC(=[NH2])N
2,1am0,AMP,B,A,GGGUUGGGAAGAAACUGUGGCACUUCGGUGCCAGCAACCC,[P@@](=O)(O)(O)OC[C@H]1O[C@@H]([C@@H]([C@@H]1O...
3,1arj,ARG,B,N,GGCAGAUCUGAGCCUGGGAGCUCUCUGCC,[NH3][C@H](C(=O)O)CCCNC(=[NH2])N
4,1eht,TEP,B,A,GGCGAUACCAGCCGAAAGGCCCUUGGCAGCGUC,Cn1c(=O)n(C)c2c(c1=O)[nH]cn2
...,...,...,...,...,...,...
1385,6yl5,SAH,NaN,K,GGUCACAACGGCUUCCUGGCGUGACCAUUGGAGCA,N[C@H](CCSC[C@H]1O[C@@H]([C@@H]([C@H]1O)O)n1cn...
1386,6ymi,AMP,Z,O,GGUCACAACGGCUUCCUGGCGUGACC,P(=O)(O)(O)OC[C@@H]1O[C@H]([C@@H]([C@H]1O)O)n1...
1387,6ymj,ADN,AA,O,GGUCACAACGGCUUCCUGGCGUGACC,OC[C@H]1O[C@@H]([C@H]([C@H]1O)O)n1cnc2c(N)ncnc12
1388,7tql,5GP,NaN,3,AGCAGAGUGGCGCAGCGGAAGCGUGCUGGGCCCAUAACCCAGAGGU...,P(=O)(=O)OC[C@@H]1O[C@@H]([C@H]([C@@H]1O)O)n1c...


## Fix SMILES

In [12]:
import pandas as pd
merged_df = pd.read_csv('pdbbind_dataset_rna/pdbbind_rna_processed_index.csv')

In [30]:
import os
from rdkit import Chem
from rdkit.Chem import rdmolops

# Set up the path to your PDB files
ligand_pdb_folder = '/home/tzutang.lin/blue_ufhpc/CoPRA/CoRSA/pdbbind_dataset_rna/parsed_ligand_pdb'

# Function to extract SMILES using RDKit
def extract_ligand_smiles(row):
    pdb_id = row['pdb_id']
    ligand_id = row['ligand_id']
    ligand_chain = row['ligand_chain']
    pdb_file_path = os.path.join(ligand_pdb_folder, f'{pdb_id}_{ligand_id}_{ligand_chain}.pdb')

    if not os.path.exists(pdb_file_path):
        print(f"Warning: PDB file {pdb_file_path} not found.")
        return None

    try:
        # Load the PDB file without sanitization
        mol = Chem.MolFromPDBFile(pdb_file_path, sanitize=False)

        if mol is None:
            print(f"Warning: RDKit could not parse {pdb_file_path}")
            return None

        # Split into fragments and attempt to find the ligand-sized fragment
        fragments = rdmolops.GetMolFrags(mol, asMols=True, sanitizeFrags=True)

        if not fragments:
            print(f"Warning: No fragments found for PDB file {pdb_file_path}")
            return None

        # Find the most suitable fragment to represent the ligand
        for frag in fragments:
            if frag.GetNumAtoms() < 100:  # Adjust the threshold if needed
                ligand_smiles = Chem.MolToSmiles(frag, canonical=True)
                if not ligand_smiles:
                    print(f"Warning: Failed to generate SMILES for fragment in file: {pdb_file_path}")
                    return None
                return ligand_smiles

        # If no suitable fragment is found
        print(f"Warning: No suitable fragment found for SMILES generation in file {pdb_file_path}")
        return None

    except Exception as e:
        print(f"Error extracting SMILES from {pdb_file_path} (PDB ID: {pdb_id}, Ligand ID: {ligand_id}, Chain: {ligand_chain}): {e}")
        return None

# Apply the function to the DataFrame
merged_df['ligand_smiles'] = merged_df.apply(extract_ligand_smiles, axis=1)


In [33]:
merged_df.to_csv('pdbbind_dataset_rna/pdbbind_rna_processed_index.csv', index=False)

In [ ]:
general_df = pd.read_csv('general_dataset/general_processed_index.csv', keep_default_na=False)

In [69]:
# Apply the function to the DataFrame
ligand_pdb_folder = '/home/tzutang.lin/blue_ufhpc/CoPRA/CoRSA/general_dataset/parsed_ligands_pdb'

# Function to extract SMILES using RDKit
def extract_ligand_smiles(row):
    pdb_id = row['pdb_id']
    ligand_id = row['ligand_id']
    ligand_chain = row['ligand_chain']
    pdb_file_path = os.path.join(ligand_pdb_folder, f'{pdb_id}_{ligand_chain}_{ligand_id}.pdb')

    if not os.path.exists(pdb_file_path):
        print(f"Warning: PDB file {pdb_file_path} not found.")
        return None

    try:
        # Load the PDB file without sanitization
        mol = Chem.MolFromPDBFile(pdb_file_path, sanitize=False)

        if mol is None:
            print(f"Warning: RDKit could not parse {pdb_file_path}")
            return None

        # Split into fragments
        fragments = rdmolops.GetMolFrags(mol, asMols=True, sanitizeFrags=True)

        if not fragments:
            print(f"Warning: No fragments found for PDB file {pdb_file_path}")
            return None

        # Select the fragment most likely to be the ligand (based on your specific logic)
        # Here we are choosing the first fragment arbitrarily; you may need a smarter heuristic
        ligand_fragment = fragments[0]

        # Generate the SMILES representation
        ligand_smiles = Chem.MolToSmiles(ligand_fragment, canonical=True)
        if not ligand_smiles:
            print(f"Warning: Failed to generate SMILES for fragment in file: {pdb_file_path}")
            return None

        return ligand_smiles

    except Exception as e:
        print(f"Error extracting SMILES from {pdb_file_path} (PDB ID: {pdb_id}, Ligand ID: {ligand_id}, Chain: {ligand_chain}): {e}")
        return None

general_df['ligand_smiles'] = general_df.apply(extract_ligand_smiles, axis=1)

In [51]:
general_df

,pdb_id,ligand_id,ligand_chain,rna_chain,rna_sequence,ligand_smiles
0,1aju,ARG,B,A,GGCCAGAUUGAGCCUGGGAGCUCUCUGGCC,[H]N([H])C(N([H])[H])N([H])C([H])([H])C([H])([...
1,1akx,ARG,B,A,GGCCAGAUUGAGCCUGGGAGCUCUCUGGCC,[H]N([H])C(N([H])[H])N([H])C([H])([H])C([H])([...
2,1am0,AMP,B,A,GGGUUGGGAAGAAACUGUGGCACUUCGGUGCCAGCAACCC,[H]O[C@@]1([H])[C@@]([H])(O[H])[C@]([H])(N2C([...
3,1arj,ARG,B,N,GGCAGAUCUGAGCCUGGGAGCUCUCUGCC,[H]N([H])C(N([H])[H])N([H])C([H])([H])C([H])([...
4,1eht,TEP,B,A,GGCGAUACCAGCCGAAAGGCCCUUGGCAGCGUC,[H]C1NC2C(C(O)N(C([H])([H])[H])C(O)N2C([H])([H...
...,...,...,...,...,...,...
1385,6yl5,SAH,NA,K,GGUCACAACGGCUUCCUGGCGUGACCAUUGGAGCA,[H]O[C@@]1([H])[C@@]([H])(O[H])[C@]([H])(N2C([...
1386,6ymi,AMP,Z,O,GGUCACAACGGCUUCCUGGCGUGACC,NC1NCNC2C1NCN2[C@@H]1O[C@H](CO[PH](O)(O)O)[C@@...
1387,6ymj,ADN,AA,O,GGUCACAACGGCUUCCUGGCGUGACC,NC1NCNC2C1NCN2[C@@H]1O[C@H](CO)[C@@H](O)[C@H]1O
1388,7tql,5GP,NA,3,AGCAGAGUGGCGCAGCGGAAGCGUGCUGGGCCCAUAACCCAGAGGU...,NC1NC(O)C2NCN([C@@H]3O[C@H](COP(O)O)[C@@H](O)[...


In [71]:
general_df.to_csv('general_dataset/general_processed_index.csv', index=False)